# 팀 코드

본 노트북은 지금까지 했던 과정을 간략히 정리한 것입니다. 참고용으로 보시기 바랍니다.

# 제출 기록

1. D-6
 - 파생 변수 생성 확인
    1. errtype을 var로 생성 : 0.83068(기각)
    2. errtype, errcode 유니크 개수: 0.83235(기각)
    3. errtype 유니크 개수: 0.83261(기각)
    

2. D-5
 - 파생 변수 생성 확인
    1. err, qual의 time을 second으로 하고 std 로 groupby : 0.83490(상승)
    2. 혜인이가 추천한 중요도 높은 errcode 추가: 0.83447(기각)
    3. quality-groupby-sum: 0.83518(상승)

3. D-4
 - 베스트 모델 확인
    1. catb: 0.83552(상승)(2위)
    2. gbc: 0.82778(기각)(3위)
    3. ~~lgbm-1: 0.83239(기각)~~

4. D-3
 - 베스트 모델 확인
    1. lgbm-2: 0.83569(상승)(1위)
    2. xtree: 0.81803(기각)(6위)
    3. rf: 0.82260(기각)(5위)

5. D-2
 - 앙상블 제출
    1. ngb: 0.82469(기각)(4위)
    2. 최고모델에 혜인이 변수 조정 버전: 	0.83746(상승)
    3. best-6: 	0.83395(기각)

6. D-1
 - 앙상블 제출
    1. best-5(xtree 제외): 0.83628(기각)
    2. best-4(xtree, rf 제외): 0.83667(기각)
    3. best-3(xtree, rf, ngb 제외): 0.83775(상승)

7. D-Day
 - 역전 재판
    1. best-2(catb, lgbm 만 사용): 0.83645(기각)
    2. ngb: 0.82906(기각)
    3. 최종 best-3 앙상블(catb, lgbm, ngb): 0.83713(기각)

In [1]:
import load_dtypes as ld
import warnings
warnings.filterwarnings(action='ignore')

import os, sys
import time
import datetime as dt

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_auc_score, roc_curve, precision_recall_curve, recall_score, precision_score

from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli
from ngboost.scores import LogScore, CRPScore
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold

In [2]:
TRAIN_P_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv'
TRAIN_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv'
TRAIN_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv'
TEST_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv'
TEST_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/sample_submission.csv'

In [3]:
%%time
train_p = ld.load_dtypes(TRAIN_P_PATH)
train_q = ld.load_dtypes(TRAIN_Q_PATH)
train_e = ld.load_dtypes(TRAIN_E_PATH)
test_q = ld.load_dtypes(TEST_Q_PATH)
test_e = ld.load_dtypes(TEST_E_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv
Wall time: 1min 17s


In [4]:
train_p.shape, train_q.shape, train_e.shape, test_q.shape, test_e.shape, submission.shape

((5429, 2),
 (828624, 16),
 (16554663, 6),
 (747972, 16),
 (16532648, 6),
 (14999, 2))

# Preprocessing

In [5]:
def preprocessing_problem(df, object_='binary'):
    """
    definition:
    train_problem 테이블을 받아서 target 값으로 변환
    1. {0, 1}의 binary로 변환
    2. {0 ~ n}의 multiclass로 변환
    """
    
    # 10001부터 24999까지의 index를 만들어줍니다.
    user_id_idx = np.array(range(10000, 25000, 1))
    
    # train_new_p라는 새로운 df를 만들고 index는 위에서 만든 user_id_idx 로 지정해줍니다.
    new_p = pd.DataFrame(index = user_id_idx)
    new_p['target'] = 0
    
    if object_ == 'binary':
        new_p.iloc[df.user_id.unique()-10000] = 1
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    elif object_ == 'multi':
        # multi는 count()로 집계를 해줍니다.
        new_p['target'] = df.groupby('user_id')['time'].count()
        new_p = new_p.fillna(0)
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    return new_p

In [6]:
train_b_p = preprocessing_problem(train_p, 'binary')
train_m_p = preprocessing_problem(train_p, 'multi')

In [7]:
def preprocessing_quality(df):
    """
    definition:
    EDA를 통해 알아낸 정보로 train_q, test_q를 정리해서 내뿜어줍니다.
    1. qaulity_3, quality_4 를 drop 합니다.(단일 value)
    2. qaulity_k 변수들을 정수로 encoding 합니다.
    """
    # 먼저 3, 4번을 drop 합니다.
    df.drop(['quality_3', 'quality_4'], axis=1, inplace=True)
    
    # qual 변수만 할당해주고, 정수로 형변환 해줍니다.
    columns = train_q.columns[train_q.columns.str.contains('quality')]
    # for문을 통해 각 column을 반복 작업해줍니다.
    for col in columns:
        try:
            if df[col].dtype == 'float32': # 기존에 float은 패스
                df[col] = df[col].fillna(-2)
            elif df[col].dtype == 'int8' or df[col].dtype == 'int16': # 기존에 int도 패스
                df[col] = df[col].fillna(-2)
                #print(col)
            else:
                df[col] = df[col].astype('object')
                # nan값이 있으면 float으로 갈 수 없으니 '-2' 으로 채워줍니다.
                df[col] = df[col].fillna('-2')
                df[col] = df[col].apply(lambda x: x.replace(',' , ''))
                df[col] = df[col].astype(np.float32)
        except:
            pass
        
    # fwver 에서 null 값이 꽤 있습니다. missing으로 채우겠습니다.
    df.fwver = df.fwver.astype('object')
    df.fwver = df.fwver.fillna('missing')
    df.fwver = df.fwver.astype('category')
        
    return df

In [8]:
train_q = preprocessing_quality(train_q)
test_q = preprocessing_quality(test_q)

In [9]:
def preprocessing_fwver(df):
    """
    definition:
    별건 아니고, e-set에 fwver 변수에서 '10' 이라는 값이 있는데, 이게 errtype이랑 겹쳐요.
    그래서 10을 -> 8.5.2 으로 바꿔줄 겁니다.
    굳이 이렇게 바꾸는 이유는, 해당 fw가 8.5.3버전과 같은 model_nm을 공유하기 때문입니다.
    """
    df.fwver = df.fwver.replace('10', '8.5.2')
        
    return df

In [10]:
train_e = preprocessing_fwver(train_e)
test_e = preprocessing_fwver(test_e)

In [11]:
def make_datetime(df):
    """
    definition:
    'time' column이 str로 되어 있으니, datetime으로 바꿔주는 함수입니다.
    다만 'time'양식이 pandas함수에 적용이 안되니, 강제로 슬라이싱해서 만들어줘야 합니다.
    
    """
    
    df['year'] = df['time'].apply(lambda x: str(x)[:4])
    df['month'] = df['time'].apply(lambda x: str(x)[4:6])
    df['day'] = df['time'].apply(lambda x: str(x)[6:8])
    df['hour'] = df['time'].apply(lambda x: str(x)[8:10])
    df['minute'] = '00' # minute을 넣어주지 않으면 datetime이 완성이 안되니, 00으로 넣어줍니다.
    
    df['time'] = pd.to_datetime(df.year + df.month + df.day + df.hour + df.minute)
    
    return df

In [12]:
%%time
train_p = make_datetime(train_p)
train_q = make_datetime(train_q)
test_q = make_datetime(test_q)
train_e = make_datetime(train_e)
test_e = make_datetime(test_e)

Wall time: 58.9 s


In [13]:
# groupby 연산에서 왜 문제가 생기나 했더니 category로 되어 있어서였습니다.

train_e['errtype'] = train_e.errtype.astype('object')
test_e['errtype'] = test_e.errtype.astype('object')

train_e['errcode'] = train_e.errcode.astype('object')
test_e['errcode'] = test_e.errcode.astype('object')

In [14]:
# fwver도 object로 잡아줍니다.

train_q.fwver = train_q.fwver.astype('object')
test_q.fwver = test_q.fwver.astype('object')

# Feature Engineering

## from Error_log

### 사용한 model의 개수

In [15]:
train_model_count = train_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()
test_model_count = test_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()

### 사용한 fwver 의 개수

In [16]:
train_fwver_count = train_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()
test_fwver_count = test_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()

### 경험한 error 의 총 개수

In [17]:
train_err_count = train_e.groupby('user_id')['errcode'].count()
test_err_count = test_e.groupby('user_id')['errcode'].count()

### 경험한 각 errtype의 value별 개수

In [18]:
# 33호
train_errcode_33 = train_e[train_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_33 = test_e[test_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']
test_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']

In [19]:
# 23호
train_errcode_23 = train_e[train_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_23 = test_e[test_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']
test_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']

In [20]:
# 34호
train_errcode_34 = train_e[train_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_34 = test_e[test_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']
test_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']

## from Quality_log

### 각 quality의 표준편차

In [21]:
train_qual_std = train_q.groupby(['user_id']).std()
test_qual_std = test_q.groupby(['user_id']).std()

new_columns = ['q_std_0', 'q_std_1', 'q_std_2', 'q_std_5', 'q_std_6', 'q_std_7', 'q_std_8', 'q_std_9', 'q_std_10','q_std_11', 'q_std_12']

train_qual_std.columns = new_columns
test_qual_std.columns = new_columns

### 기록한 quality log의 개수(12개당 1번)

In [22]:
train_qual_log = train_q.groupby('user_id')['time'].count()/12
test_qual_log = test_q.groupby('user_id')['time'].count()/12

### quality당 순수 개수

In [23]:
# 값을 만들기 위해 새로운 df를 받아옵니다.
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

# 0 값을 전부 nan 값으로 바꿔줍니다.
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan

# 필요없는 변수들을 버려줍니다.
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

# 그룹바이 카운트 해줍니다.
train_qual_counts = train_q_temp.groupby('user_id').count()
test_qual_counts = test_q_temp.groupby('user_id').count()

# 겹치는 컬럼명을 바꿔줍니다.
train_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']
test_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']

### quality당 음수, 0에 대한 count

In [24]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [25]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [26]:
# 0 값만 count를 위해서 음수와 양수를 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan

In [27]:
train_qual_zeroCount = train_q_temp.groupby('user_id').count()
test_qual_zeroCount = test_q_temp.groupby('user_id').count()

In [28]:
new_columns = ['q_z_c_0', 'q_z_c_1', 'q_z_c_2', 'q_z_c_5', 'q_z_c_6', 'q_z_c_7', 'q_z_c_8', 'q_z_c_9', 'q_z_c_10','q_z_c_11', 'q_z_c_12']

train_qual_zeroCount.columns = new_columns
test_qual_zeroCount.columns = new_columns

In [29]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [30]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [31]:
# 음수 값만 count를 위해서 음수와 0을 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan

In [32]:
train_qual_negaCount = train_q_temp.groupby('user_id').count()
test_qual_negaCount = test_q_temp.groupby('user_id').count()

In [33]:
new_columns = ['q_n_c_0', 'q_n_c_1', 'q_n_c_2', 'q_n_c_5', 'q_n_c_6', 'q_n_c_7', 'q_n_c_8', 'q_n_c_9', 'q_n_c_10','q_n_c_11', 'q_n_c_12']

train_qual_negaCount.columns = new_columns
test_qual_negaCount.columns = new_columns

## Adding

### time에 대한 유저별 표준편차

In [34]:
import time, datetime

def time_to_seconds(x):
    return time.mktime(x.timetuple())

In [35]:
%%time
train_e['time_sec'] = train_e.time.apply(lambda x: time_to_seconds(x))
test_e['time_sec'] = test_e.time.apply(lambda x: time_to_seconds(x))
train_q['time_sec'] = train_q.time.apply(lambda x: time_to_seconds(x))
test_q['time_sec'] = test_q.time.apply(lambda x: time_to_seconds(x))

Wall time: 2min 7s


In [36]:
%%time
train_err_timestd = train_e.groupby(['user_id'])['time_sec'].std()
test_err_timestd = test_e.groupby(['user_id'])['time_sec'].std()
train_err_timestd = train_err_timestd.rename(level = 0, index = 't_e_std') 
test_err_timestd = test_err_timestd.rename(level = 0, index = 't_e_std') 

Wall time: 481 ms


In [37]:
%%time
train_qual_timestd = (train_q[['user_id', 'time_sec']].drop_duplicates()).groupby(['user_id']).std()
test_qual_timestd = (test_q[['user_id', 'time_sec']].drop_duplicates()).groupby(['user_id']).std()
train_qual_timestd.columns = ['t_q_std']
test_qual_timestd.columns = ['t_q_std']

Wall time: 87 ms


### quality를 sum으로 groupby

In [38]:
train_each_quality_sum = train_q.groupby('user_id').sum().loc[:, 'quality_0':'quality_12']
test_each_quality_sum = test_q.groupby('user_id').sum().loc[:, 'quality_0':'quality_12']

quality_sum_colnms = ['quality_0_sum', 'quality_1_sum', 'quality_2_sum', 'quality_5_sum', 'quality_6_sum', 
                      'quality_7_sum', 'quality_8_sum', 'quality_9_sum','quality_10_sum', 'quality_11_sum', 
                      'quality_12_sum']

train_each_quality_sum.columns = quality_sum_colnms
test_each_quality_sum.columns = quality_sum_colnms

### 죽지도 않고 또 온 31호

In [39]:
# 혜인이 다시 제안한 조건부 31호.
train_errcode_31 = train_e[train_e.errtype == 31][['user_id', 'errcode', 'time']].\
                            groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_31 = test_e[test_e.errtype == 31][['user_id', 'errcode', 'time']].\
                            groupby(['user_id', 'errcode']).count().unstack().fillna(0)
train_errcode_31.columns = ['err_31_0', 'err_31_1']
test_errcode_31.columns =['err_31_0', 'err_31_1']

### day max를 mean으로 나눈 것

In [40]:
%%time
train_e['day_2'] = train_e['time'].apply(lambda x: str(x)[:10])

train_meanDay = (train_e
                 .groupby(['user_id','day_2'])['day_2']
                 .count()
                 .unstack()
                 .fillna(0)
                 .loc[:, '2020-11-01':'2020-11-30']
                 .mean(axis=1))

train_maxDay = (train_e
                .groupby(['user_id','day_2'])['day_2']
                .count()
                .unstack()
                .fillna(0)
                .loc[:, '2020-11-01':'2020-11-30']
                .max(axis=1))

train_maxBymean = pd.Series(data = np.array(train_maxDay) / np.array(train_meanDay),
                            index = train_e.user_id.unique(),
                            name = 'mbm')

Wall time: 1min 15s


In [41]:
%%time
test_e['day_2'] = test_e['time'].apply(lambda x: str(x)[:10])

test_meanDay = (test_e
                .groupby(['user_id','day_2'])['day_2']
                .count()
                .unstack()
                .fillna(0)
                .loc[:, '2020-11-01':'2020-11-30']
                .mean(axis=1))

test_maxDay = (test_e
               .groupby(['user_id','day_2'])['day_2']
               .count()
               .unstack()
               .fillna(0)
               .loc[:, '2020-11-01':'2020-11-30']
               .max(axis=1))

test_maxBymean = pd.Series(data = np.array(test_maxDay) / np.array(test_meanDay),
                           index = test_e.user_id.unique(),
                           name = 'mbm')

Wall time: 1min 14s


제작 중

In [43]:
# ## 시간 타입으로 변환
train_e.hour = train_e.hour.astype(np.int16)
test_e.hour = test_e.hour.astype(np.int16)

train_e.loc[train_e.hour < 12, 'AM'] = 'AM'
train_e.loc[train_e.hour >= 12, 'PM'] = 'PM'

test_e.loc[test_e.hour < 12, 'AM'] = 'AM'
test_e.loc[test_e.hour >= 12, 'PM'] = 'PM'

train_err_am = train_e.groupby(['user_id', 'AM'])['AM'].count().unstack()
train_err_pm = train_e.groupby(['user_id', 'PM'])['PM'].count().unstack()
test_err_am = test_e.groupby(['user_id', 'AM'])['AM'].count().unstack()
test_err_pm = test_e.groupby(['user_id', 'PM'])['PM'].count().unstack()

# Train & Predict

### Base set

In [44]:
# errtype을 유저별로 카운트 해줍니다.

X = train_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)
y = test_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)

X.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']
y.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']

In [45]:
# 하나 사라진 유저를 채워주는 코드입니다.
y = y.reindex(pd.RangeIndex(y.index.max() + 1)).ffill(0)[30000:]

In [46]:
X.shape, y.shape

((15000, 41), (14999, 41))

### 1차 합병

In [47]:
X = pd.concat([X,
               train_err_count, # 유저가 기록한 총 err수
               train_fwver_count, # 유저가 사용한 fw수
               train_model_count, # 유저가 사용한 model 수
               train_qual_std, # 각 퀄리티에 대한 유저별 편차
               train_qual_log, # 유저별 퀄리티 로그의 수
               train_errcode_23, # 23호 에러타입의 밸류별 개수
               train_errcode_33, # 33호 상동
               train_errcode_34, # 34호 상동
               train_qual_counts, # 각 퀄리티에서 0을 제외한 카운트
               train_qual_negaCount, # 각 퀄리티에 대해 음수만 카운트
               train_qual_zeroCount # 각 퀄리티에 대해 0.만 카운트
              ], axis=1).fillna(0)
X.shape

(15000, 106)

In [48]:
y = pd.concat([y,
               test_err_count,
               test_fwver_count,
               test_model_count,
               test_qual_std,
               test_qual_log,
               test_errcode_23,
               test_errcode_33,
               test_errcode_34,
               test_qual_counts,
               test_qual_negaCount,
               test_qual_zeroCount
              ], axis=1).fillna(0)
y.shape

(14999, 106)

### 3차 합격생들

In [49]:
X = pd.concat([X, train_err_timestd, train_qual_timestd, train_each_quality_sum, train_errcode_31, train_maxBymean], axis=1).fillna(0)
y = pd.concat([y, test_err_timestd, test_qual_timestd, test_each_quality_sum, test_errcode_31, test_maxBymean], axis=1).fillna(0)

X.drop(['et_20', 'et_36'], axis=1, inplace = True)
y.drop(['et_20', 'et_36'], axis=1, inplace = True)

X.shape, y.shape

((15000, 120), (14999, 120))

### 3차 면접중

In [50]:
X_temp = pd.concat([X, train_err_am, train_err_pm], axis=1).fillna(0)
y_temp = pd.concat([y, test_err_am, test_err_pm], axis=1).fillna(0)

X_temp.shape, y_temp.shape

((15000, 122), (14999, 122))

혜인이 막판 뒤집기

In [176]:
#3. [err] errtype
train_err_unique_errtype = train_e[['user_id', 'errtype']].drop_duplicates().groupby('user_id').count()
test_err_unique_errtype = test_e[['user_id', 'errtype']].drop_duplicates().groupby('user_id').count()
train_err_unique_errtype.columns = ['err_unique_errtype']
test_err_unique_errtype.columns = ['err_unique_errtype']

In [182]:
X_temp2 = pd.concat([X, train_err_unique_errtype], axis=1).fillna(0)
y_temp2 = pd.concat([y, test_err_unique_errtype], axis=1).fillna(0)

X_temp2.drop(['et_4'], axis=1, inplace = True)
y_temp2.drop(['et_4'], axis=1, inplace = True)

X_temp2.shape, y_temp2.shape

((15000, 120), (14999, 120))

In [183]:
X_temp3 = pd.concat([X, train_err_am, train_err_pm, train_err_unique_errtype], axis=1).fillna(0)
y_temp3 = pd.concat([y, test_err_am, test_err_pm, test_err_unique_errtype], axis=1).fillna(0)

X_temp3.drop(['et_4'], axis=1, inplace = True)
y_temp3.drop(['et_4'], axis=1, inplace = True)

X_temp3.shape, y_temp3.shape

((15000, 122), (14999, 122))

In [209]:
X

,et_1,et_2,et_3,et_4,et_5,et_6,et_7,et_8,et_9,et_10,et_11,et_12,et_13,et_14,et_15,et_16,et_17,et_18,et_19,et_21,et_22,et_23,et_24,et_25,et_26,et_27,et_28,et_30,et_31,et_32,et_33,et_34,et_35,et_37,et_38,et_39,et_40,et_41,et_42,errcode,fwver,model_nm,q_std_0,q_std_1,q_std_2,q_std_5,q_std_6,q_std_7,q_std_8,q_std_9,q_std_10,q_std_11,q_std_12,time,UNKNOWN,ACTIVE,connLMP,connESTA,connTO,connLOCAL,STANDBY,TERMINATE,err_33_1,err_33_2,err_33_3,err_34_1,err_34_2,err_34_3,err_34_4,err_34_5,err_34_6,q_c_0,q_c_1,q_c_2,q_c_5,q_c_6,q_c_7,q_c_8,q_c_9,q_c_10,q_c_11,q_c_12,q_n_c_0,q_n_c_1,q_n_c_2,q_n_c_5,q_n_c_6,q_n_c_7,q_n_c_8,q_n_c_9,q_n_c_10,q_n_c_11,q_n_c_12,q_z_c_0,q_z_c_1,q_z_c_2,q_z_c_5,q_z_c_6,q_z_c_7,q_z_c_8,q_z_c_9,q_z_c_10,q_z_c_11,q_z_c_12,t_e_std,t_q_std,quality_0_sum,quality_1_sum,quality_2_sum,quality_5_sum,quality_6_sum,quality_7_sum,quality_8_sum,quality_9_sum,quality_10_sum,quality_11_sum,quality_12_sum,err_31_0,err_31_1,mbm
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317,1,1,0.000000,0.000000,0.000000,1.793709,0.000000,0.000000,0.0,0.000000,2.043016,0.000000,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,24.0,24.0,22.0,24.0,24.0,24.0,24.0,0.0,24.0,24.0,760181.244016,91641.038842,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,1.892744
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,2365,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0,625.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,696650.374750,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.0,124.0,18.418605
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306,1,1,0.251312,0.143576,0.177396,1.037404,4.495417,14.547924,0.0,0.332455,2.327185,0.143576,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,14.0,5.0,24.0,0.0,12.0,84.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,93.0,94.0,93.0,82.0,91.0,72.0,96.0,84.0,12.0,94.0,96.0,727335.561994,842035.085798,0.0,-2.0,-1.0,29.0,44.0,552.0,0.0,12.0,372.0,-2.0,0.0,0.0,0.0,1.666667
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,306,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,12.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,759965.723107,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,32.0,2.352941
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,777,2,1,0.282330,0.282330,0.282330,0.583592,17.778784,44.435591,0.0,0.000000,1.021508,0.282330,0.0,2.0,0.0,7.0,0.0,1.0,104.0,0.0,5.0,2.0,0.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,5.0,3.0,12.0,0.0,0.0,24.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,22.0,22.0,22.0,19.0,21.0,12.0,24.0,24.0,0.0,22.0,24.0,733973.687056,94186.623254,-2.0,-2.0,-2.0,2.0,85.0,1044.0,0.0,0.0,48.0,-2.0,0.0,95.0,81.0,3.938224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [210]:
X_temp

,et_1,et_2,et_3,et_4,et_5,et_6,et_7,et_8,et_9,et_10,et_11,et_12,et_13,et_14,et_15,et_16,et_17,et_18,et_19,et_21,et_22,et_23,et_24,et_25,et_26,et_27,et_28,et_30,et_31,et_32,et_33,et_34,et_35,et_37,et_38,et_39,et_40,et_41,et_42,errcode,fwver,model_nm,q_std_0,q_std_1,q_std_2,q_std_5,q_std_6,q_std_7,q_std_8,q_std_9,q_std_10,q_std_11,q_std_12,time,UNKNOWN,ACTIVE,connLMP,connESTA,connTO,connLOCAL,STANDBY,TERMINATE,err_33_1,err_33_2,err_33_3,err_34_1,err_34_2,err_34_3,err_34_4,err_34_5,err_34_6,q_c_0,q_c_1,q_c_2,q_c_5,q_c_6,q_c_7,q_c_8,q_c_9,q_c_10,q_c_11,q_c_12,q_n_c_0,q_n_c_1,q_n_c_2,q_n_c_5,q_n_c_6,q_n_c_7,q_n_c_8,q_n_c_9,q_n_c_10,q_n_c_11,q_n_c_12,q_z_c_0,q_z_c_1,q_z_c_2,q_z_c_5,q_z_c_6,q_z_c_7,q_z_c_8,q_z_c_9,q_z_c_10,q_z_c_11,q_z_c_12,t_e_std,t_q_std,quality_0_sum,quality_1_sum,quality_2_sum,quality_5_sum,quality_6_sum,quality_7_sum,quality_8_sum,quality_9_sum,quality_10_sum,quality_11_sum,quality_12_sum,err_31_0,err_31_1,mbm,AM,PM
10000,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317,1,1,0.000000,0.000000,0.000000,1.793709,0.000000,0.000000,0.0,0.000000,2.043016,0.000000,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,24.0,24.0,22.0,24.0,24.0,24.0,24.0,0.0,24.0,24.0,760181.244016,91641.038842,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,1.892744,224.0,93.0
10001,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,2365,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0,625.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,696650.374750,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.0,124.0,18.418605,433.0,1932.0
10002,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306,1,1,0.251312,0.143576,0.177396,1.037404,4.495417,14.547924,0.0,0.332455,2.327185,0.143576,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,14.0,5.0,24.0,0.0,12.0,84.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,93.0,94.0,93.0,82.0,91.0,72.0,96.0,84.0,12.0,94.0,96.0,727335.561994,842035.085798,0.0,-2.0,-1.0,29.0,44.0,552.0,0.0,12.0,372.0,-2.0,0.0,0.0,0.0,1.666667,148.0,158.0
10003,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,306,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,12.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,759965.723107,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,32.0,2.352941,189.0,117.0
10004,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,777,2,1,0.282330,0.282330,0.282330,0.583592,17.778784,44.435591,0.0,0.000000,1.021508,0.282330,0.0,2.0,0.0,7.0,0.0,1.0,104.0,0.0,5.0,2.0,0.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,5.0,3.0,12.0,0.0,0.0,24.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,22.0,22.0,22.0,19.0,21.0,12.0,24.0,24.0,0.0,22.0,24.0,733973.687056,94186.623254,-2.0,-2.0,-2.0,2.0,85.0,1044.0,0.0,0.0,48.0,-2.0,0.0,95.0,81.0,3.938224,321.0,456.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [211]:
xX_temp2

,et_1,et_2,et_3,et_5,et_6,et_7,et_8,et_9,et_10,et_11,et_12,et_13,et_14,et_15,et_16,et_17,et_18,et_19,et_21,et_22,et_23,et_24,et_25,et_26,et_27,et_28,et_30,et_31,et_32,et_33,et_34,et_35,et_37,et_38,et_39,et_40,et_41,et_42,errcode,fwver,model_nm,q_std_0,q_std_1,q_std_2,q_std_5,q_std_6,q_std_7,q_std_8,q_std_9,q_std_10,q_std_11,q_std_12,time,UNKNOWN,ACTIVE,connLMP,connESTA,connTO,connLOCAL,STANDBY,TERMINATE,err_33_1,err_33_2,err_33_3,err_34_1,err_34_2,err_34_3,err_34_4,err_34_5,err_34_6,q_c_0,q_c_1,q_c_2,q_c_5,q_c_6,q_c_7,q_c_8,q_c_9,q_c_10,q_c_11,q_c_12,q_n_c_0,q_n_c_1,q_n_c_2,q_n_c_5,q_n_c_6,q_n_c_7,q_n_c_8,q_n_c_9,q_n_c_10,q_n_c_11,q_n_c_12,q_z_c_0,q_z_c_1,q_z_c_2,q_z_c_5,q_z_c_6,q_z_c_7,q_z_c_8,q_z_c_9,q_z_c_10,q_z_c_11,q_z_c_12,t_e_std,t_q_std,quality_0_sum,quality_1_sum,quality_2_sum,quality_5_sum,quality_6_sum,quality_7_sum,quality_8_sum,quality_9_sum,quality_10_sum,quality_11_sum,quality_12_sum,err_31_0,err_31_1,mbm,err_unique_errtype
10000,0.0,0.0,8.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317,1,1,0.000000,0.000000,0.000000,1.793709,0.000000,0.000000,0.0,0.000000,2.043016,0.000000,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,24.0,24.0,22.0,24.0,24.0,24.0,24.0,0.0,24.0,24.0,760181.244016,91641.038842,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,1.892744,14
10001,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,2365,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0,625.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,696650.374750,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.0,124.0,18.418605,26
10002,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306,1,1,0.251312,0.143576,0.177396,1.037404,4.495417,14.547924,0.0,0.332455,2.327185,0.143576,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,14.0,5.0,24.0,0.0,12.0,84.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,93.0,94.0,93.0,82.0,91.0,72.0,96.0,84.0,12.0,94.0,96.0,727335.561994,842035.085798,0.0,-2.0,-1.0,29.0,44.0,552.0,0.0,12.0,372.0,-2.0,0.0,0.0,0.0,1.666667,16
10003,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,306,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,12.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,759965.723107,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,32.0,2.352941,18
10004,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,777,2,1,0.282330,0.282330,0.282330,0.583592,17.778784,44.435591,0.0,0.000000,1.021508,0.282330,0.0,2.0,0.0,7.0,0.0,1.0,104.0,0.0,5.0,2.0,0.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,5.0,3.0,12.0,0.0,0.0,24.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,22.0,22.0,22.0,19.0,21.0,12.0,24.0,24.0,0.0,22.0,24.0,733973.687056,94186.623254,-2.0,-2.0,-2.0,2.0,85.0,1044.0,0.0,0.0,48.0,-2.0,0.0,95.0,81.0,3.938224,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [212]:
X_temp3

,et_1,et_2,et_3,et_5,et_6,et_7,et_8,et_9,et_10,et_11,et_12,et_13,et_14,et_15,et_16,et_17,et_18,et_19,et_21,et_22,et_23,et_24,et_25,et_26,et_27,et_28,et_30,et_31,et_32,et_33,et_34,et_35,et_37,et_38,et_39,et_40,et_41,et_42,errcode,fwver,model_nm,q_std_0,q_std_1,q_std_2,q_std_5,q_std_6,q_std_7,q_std_8,q_std_9,q_std_10,q_std_11,q_std_12,time,UNKNOWN,ACTIVE,connLMP,connESTA,connTO,connLOCAL,STANDBY,TERMINATE,err_33_1,err_33_2,err_33_3,err_34_1,err_34_2,err_34_3,err_34_4,err_34_5,err_34_6,q_c_0,q_c_1,q_c_2,q_c_5,q_c_6,q_c_7,q_c_8,q_c_9,q_c_10,q_c_11,q_c_12,q_n_c_0,q_n_c_1,q_n_c_2,q_n_c_5,q_n_c_6,q_n_c_7,q_n_c_8,q_n_c_9,q_n_c_10,q_n_c_11,q_n_c_12,q_z_c_0,q_z_c_1,q_z_c_2,q_z_c_5,q_z_c_6,q_z_c_7,q_z_c_8,q_z_c_9,q_z_c_10,q_z_c_11,q_z_c_12,t_e_std,t_q_std,quality_0_sum,quality_1_sum,quality_2_sum,quality_5_sum,quality_6_sum,quality_7_sum,quality_8_sum,quality_9_sum,quality_10_sum,quality_11_sum,quality_12_sum,err_31_0,err_31_1,mbm,AM,PM,err_unique_errtype
10000,0.0,0.0,8.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317,1,1,0.000000,0.000000,0.000000,1.793709,0.000000,0.000000,0.0,0.000000,2.043016,0.000000,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,24.0,24.0,22.0,24.0,24.0,24.0,24.0,0.0,24.0,24.0,760181.244016,91641.038842,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,1.892744,224.0,93.0,14
10001,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,2365,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,126.0,0.0,0.0,0.0,0.0,625.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,696650.374750,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.0,124.0,18.418605,433.0,1932.0,26
10002,0.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306,1,1,0.251312,0.143576,0.177396,1.037404,4.495417,14.547924,0.0,0.332455,2.327185,0.143576,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,3.0,14.0,5.0,24.0,0.0,12.0,84.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,93.0,94.0,93.0,82.0,91.0,72.0,96.0,84.0,12.0,94.0,96.0,727335.561994,842035.085798,0.0,-2.0,-1.0,29.0,44.0,552.0,0.0,12.0,372.0,-2.0,0.0,0.0,0.0,1.666667,148.0,158.0,16
10003,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,306,2,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,12.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,759965.723107,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,32.0,2.352941,189.0,117.0,18
10004,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,777,2,1,0.282330,0.282330,0.282330,0.583592,17.778784,44.435591,0.0,0.000000,1.021508,0.282330,0.0,2.0,0.0,7.0,0.0,1.0,104.0,0.0,5.0,2.0,0.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,5.0,3.0,12.0,0.0,0.0,24.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,22.0,22.0,22.0,19.0,21.0,12.0,24.0,24.0,0.0,22.0,24.0,733973.687056,94186.623254,-2.0,-2.0,-2.0,2.0,85.0,1044.0,0.0,0.0,48.0,-2.0,0.0,95.0,81.0,3.938224,321.0,456.0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

## Model

In [103]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------

### Tuned Catb

In [265]:
params = {
        'nan_mode': 'Min',
        'eval_metric': 'Logloss',
        'iterations': 10000,
        'sampling_frequency': 'PerTree',
        'leaf_estimation_method': 'Newton',
        'grow_policy': 'SymmetricTree',
        'penalties_coefficient': 1,
        'boosting_type': 'Plain',
        'model_shrink_mode': 'Constant',
        'feature_border_type': 'GreedyLogSum',
        'l2_leaf_reg': 3,
        'random_strength': 1,
        'rsm': 1,
        'boost_from_average': False,
        'model_size_reg': 0.5,
        'subsample': 0.800000011920929,
        'use_best_model': False,
        'class_names': [0, 1],
        'random_seed': 2584,
        'depth': 6,
        'posterior_sampling': False,
        'border_count': 254,
        'classes_count': 0,
        'auto_class_weights': 'None',
        'sparse_features_conflict_fraction': 0,
        'leaf_estimation_backtracking': 'AnyImprovement',
        'best_model_min_trees': 1,
        'model_shrink_rate': 0,
        'min_data_in_leaf': 1,
        'loss_function': 'Logloss',
        'learning_rate': 0.028116999194025993,
        'score_function': 'Cosine',
        'task_type': 'CPU',
        'leaf_estimation_iterations': 10,
        'bootstrap_type': 'MVS',
        'max_leaves': 64
        }

catb = CatBoostClassifier(**params, verbose=0)

In [266]:
def catb_fold_train_pred(train_x, train_y, N_SPLIT=5):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=N_SPLIT, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = catb
        model.fit(X, y,
                  eval_set=(valid_x, valid_y),
                  early_stopping_rounds = 300)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

이터레이션 10000, 얼리스토핑 셋팅

In [270]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = catb_fold_train_pred(X_temp, train_b_p.target, 10)
print(np.mean(auc_scores))

0.7338


In [271]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_catb = np.mean(pred_y_list, axis = 0)

In [272]:
pred_ensemble_catb

array([[0.91969449],
       [0.23163335],
       [0.51476496],
       ...,
       [0.68887411],
       [0.9339568 ],
       [0.41309126]])

### Tuned LGBM

In [273]:
# 기존 lgbm
def lgbm_fold_train_pred(train_x, train_y, N_SPLIT=5):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'boosting_type':'gbdt', 
                    'class_weight':None,
                    'colsample_bytree':1.0,
                    'importance_type':'split',
                    'learning_rate':0.1,
                    'max_depth':-1,
                    'min_child_samples':20,
                    'min_child_weight':0.001,
                    'min_split_gain':0.0,
                    'n_estimators':100,
                    'n_jobs':-1,
                    'num_leaves':31,
                    'objective':None,
                    'random_state':2584,
                    'reg_alpha':0.0,
                    'reg_lambda':0.0,
                    'silent':True,
                    'subsample':1.0,
                    'subsample_for_bin':200000,
                    'subsample_freq':0
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=N_SPLIT, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = LGBMClassifier(**params)
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [278]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = lgbm_fold_train_pred(X, train_b_p.target, 10)
print(np.mean(auc_scores))

0.73565


In [279]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_lgbm = np.mean(pred_y_list, axis = 0)

In [280]:
pred_ensemble_lgbm

array([[0.93819144],
       [0.24549661],
       [0.44315962],
       ...,
       [0.76825373],
       [0.90830821],
       [0.42843415]])

### Tuned NGB

In [242]:
ngbc = NGBClassifier(
                        Dist = k_categorical(2),
                        n_estimators = 10000,
                        learning_rate = 0.027,
                        random_state=42
                    )

In [259]:
def ngb_fold_train_pred(train_x, train_y, N_SPLIT=5):

    models = []
    recalls = []
    precisions =[]
    auc_scores = []
    threshold = 0.5

    k_fold = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
    
    ngbc = NGBClassifier(
                        Dist = k_categorical(2),
                        n_estimators = 10000,
                        learning_rate = 0.027,
                        random_state = 42,
                        col_sample = 0.6780,
                        minibatch_frac = 0.92069
                    )
    
    for train_idx, val_idx in k_fold.split(train_x, train_y):
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx].astype(np.int)
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx].astype(np.int)

        model = ngbc
        
        model.fit(X, y,
                  X_val=valid_x, Y_val = valid_y,
                  early_stopping_rounds = 100)
        
        valid_prob = model.predict_proba(valid_x)[:,1]
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        recall = recall_score(valid_y, valid_pred)
        precision = precision_score(valid_y, valid_pred)
        auc_score = roc_auc_score(valid_y, valid_prob)

        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

    return models, auc_scores, recalls, precisions

In [233]:
def ngb_fold_train_pred(train_x, train_y, N_SPLIT=5):

    models = []
    recalls = []
    precisions =[]
    auc_scores = []
    threshold = 0.5

    k_fold = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
    
    ngbc = NGBClassifier(
                        Dist = k_categorical(2),
                        n_estimators = 1000,
                        learning_rate = 0.02,
                        random_state = 41
                    )
    
    for train_idx, val_idx in k_fold.split(train_x, train_y):
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx].astype(np.int)
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx].astype(np.int)

        model = ngbc
        
        model.fit(X, y,
                  X_val=valid_x, Y_val = valid_y,
                  early_stopping_rounds = 3)
        
        valid_prob = model.predict_proba(valid_x)[:,1]
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        recall = recall_score(valid_y, valid_pred)
        precision = precision_score(valid_y, valid_pred)
        auc_score = roc_auc_score(valid_y, valid_prob)

        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

    return models, auc_scores, recalls, precisions

검증 중

In [220]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp2, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6217 scale=2.0000 norm=4.0000
[iter 100] loss=0.4556 val_loss=0.4607 scale=2.0000 norm=3.7075
[iter 200] loss=0.4453 val_loss=0.4605 scale=1.0000 norm=1.8792
== Early stopping achieved.
== Best iteration / VAL138 (val_loss=0.4590)
[iter 0] loss=0.4450 val_loss=0.4535 scale=0.2500 norm=0.4829
[iter 100] loss=0.4395 val_loss=0.4545 scale=2.0000 norm=3.8185
== Early stopping achieved.
== Best iteration / VAL1 (val_loss=0.4535)
[iter 0] loss=0.4434 val_loss=0.4183 scale=2.0000 norm=3.8558
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4183)
[iter 0] loss=0.4399 val_loss=0.4312 scale=1.0000 norm=1.9124
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4312)
[iter 0] loss=0.4370 val_loss=0.4327 scale=0.5000 norm=0.9676
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4327)
[iter 0] loss=0.4354 val_loss=0.4294 scale=1.0000 norm=1.9080
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4294)
[iter 0]

In [221]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp3, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6218 scale=2.0000 norm=4.0000
[iter 100] loss=0.4563 val_loss=0.4633 scale=1.0000 norm=1.8548
[iter 200] loss=0.4474 val_loss=0.4631 scale=1.0000 norm=1.8769
== Early stopping achieved.
== Best iteration / VAL133 (val_loss=0.4621)
[iter 0] loss=0.4477 val_loss=0.4540 scale=1.0000 norm=1.9126
[iter 100] loss=0.4410 val_loss=0.4535 scale=0.5000 norm=0.9495
[iter 200] loss=0.4382 val_loss=0.4541 scale=0.5000 norm=0.9494
== Early stopping achieved.
== Best iteration / VAL109 (val_loss=0.4534)
[iter 0] loss=0.4421 val_loss=0.4182 scale=1.0000 norm=1.9185
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4182)
[iter 0] loss=0.4394 val_loss=0.4294 scale=0.2500 norm=0.4789
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4294)
[iter 0] loss=0.4366 val_loss=0.4365 scale=1.0000 norm=1.9179
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4365)
[iter 0] loss=0.4361 val_loss=0.4294 scale=0.5000 norm=0.9564
[iter 100] lo

In [222]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp2, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6226 scale=2.0000 norm=4.0000
[iter 100] loss=0.4556 val_loss=0.4722 scale=2.0000 norm=3.7185
[iter 200] loss=0.4452 val_loss=0.4724 scale=2.0000 norm=3.7581
== Early stopping achieved.
== Best iteration / VAL148 (val_loss=0.4713)
[iter 0] loss=0.4533 val_loss=0.4336 scale=1.0000 norm=2.0958
[iter 100] loss=0.4435 val_loss=0.4349 scale=1.0000 norm=1.9048
== Early stopping achieved.
== Best iteration / VAL30 (val_loss=0.4330)
[iter 0] loss=0.4430 val_loss=0.4335 scale=0.2500 norm=0.5009
[iter 100] loss=0.4378 val_loss=0.4363 scale=0.5000 norm=0.9514
== Early stopping achieved.
== Best iteration / VAL14 (val_loss=0.4334)
[iter 0] loss=0.4395 val_loss=0.4291 scale=0.5000 norm=0.9660
[iter 100] loss=0.4366 val_loss=0.4295 scale=0.0010 norm=0.0019
== Early stopping achieved.
== Best iteration / VAL5 (val_loss=0.4291)
[iter 0] loss=0.4326 val_loss=0.4456 scale=1.0000 norm=1.9054
[iter 100] loss=0.4306 val_loss=0.4462 scale=0.0010 norm=0.0019
== Early stopping 

In [223]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp3, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6227 scale=2.0000 norm=4.0000
[iter 100] loss=0.4547 val_loss=0.4730 scale=1.0000 norm=1.8514
[iter 200] loss=0.4465 val_loss=0.4724 scale=0.0010 norm=0.0018
== Early stopping achieved.
== Best iteration / VAL138 (val_loss=0.4716)
[iter 0] loss=0.4555 val_loss=0.4361 scale=0.5000 norm=0.9728
[iter 100] loss=0.4456 val_loss=0.4371 scale=1.0000 norm=1.9028
== Early stopping achieved.
== Best iteration / VAL39 (val_loss=0.4356)
[iter 0] loss=0.4450 val_loss=0.4365 scale=0.1250 norm=0.2607
[iter 100] loss=0.4395 val_loss=0.4405 scale=0.5000 norm=0.9533
== Early stopping achieved.
== Best iteration / VAL16 (val_loss=0.4364)
[iter 0] loss=0.4415 val_loss=0.4296 scale=0.2500 norm=0.4861
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4296)
[iter 0] loss=0.4339 val_loss=0.4465 scale=1.0000 norm=1.9034
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4465)
0.8350595000000001


이터레이션 증가, 얼리스토핑 추가

In [260]:
# 최고최고최고최괴최괴ㅚ괴최괴고치ㅚ괴최괴
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6366 val_loss=0.6225 scale=2.0000 norm=4.0002
[iter 100] loss=0.4543 val_loss=0.4596 scale=1.0000 norm=1.8490
[iter 200] loss=0.4426 val_loss=0.4563 scale=1.0000 norm=1.8717
== Early stopping achieved.
== Best iteration / VAL179 (val_loss=0.4561)
[iter 0] loss=0.4417 val_loss=0.4442 scale=0.2500 norm=0.4854
[iter 100] loss=0.4330 val_loss=0.4441 scale=1.0000 norm=1.8900
== Early stopping achieved.
== Best iteration / VAL59 (val_loss=0.4440)
[iter 0] loss=0.4375 val_loss=0.4093 scale=0.5000 norm=0.9573
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4093)
[iter 0] loss=0.4277 val_loss=0.4194 scale=0.5000 norm=0.9386
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4194)
[iter 0] loss=0.4265 val_loss=0.4263 scale=0.1250 norm=0.2388
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4263)
[iter 0] loss=0.4281 val_loss=0.4214 scale=0.5000 norm=0.9495
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4214)
[iter 0

In [252]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6366 val_loss=0.6226 scale=2.0000 norm=4.0002
[iter 100] loss=0.4551 val_loss=0.4609 scale=1.0000 norm=1.8467
[iter 200] loss=0.4415 val_loss=0.4578 scale=0.5000 norm=0.9347
== Early stopping achieved.
== Best iteration / VAL172 (val_loss=0.4571)
[iter 0] loss=0.4386 val_loss=0.4447 scale=0.2500 norm=0.4736
[iter 100] loss=0.4323 val_loss=0.4445 scale=2.0000 norm=3.7631
== Early stopping achieved.
== Best iteration / VAL33 (val_loss=0.4439)
[iter 0] loss=0.4353 val_loss=0.4120 scale=0.5000 norm=0.9468
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4120)
[iter 0] loss=0.4329 val_loss=0.4205 scale=1.0000 norm=1.8872
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4205)
[iter 0] loss=0.4281 val_loss=0.4292 scale=0.2500 norm=0.4760
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4292)
[iter 0] loss=0.4274 val_loss=0.4215 scale=0.5000 norm=0.9407
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4215)
[iter 0

In [226]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6232 scale=2.0000 norm=4.0000
[iter 100] loss=0.4544 val_loss=0.4690 scale=1.0000 norm=1.8452
[iter 200] loss=0.4437 val_loss=0.4680 scale=1.0000 norm=1.8708
== Early stopping achieved.
== Best iteration / VAL144 (val_loss=0.4673)
[iter 0] loss=0.4515 val_loss=0.4300 scale=0.2500 norm=0.5121
[iter 100] loss=0.4428 val_loss=0.4320 scale=0.5000 norm=0.9536
== Early stopping achieved.
== Best iteration / VAL11 (val_loss=0.4300)
[iter 0] loss=0.4416 val_loss=0.4351 scale=0.2500 norm=0.4923
[iter 100] loss=0.4368 val_loss=0.4379 scale=0.5000 norm=0.9513
== Early stopping achieved.
== Best iteration / VAL3 (val_loss=0.4351)
[iter 0] loss=0.4391 val_loss=0.4287 scale=1.0000 norm=1.9265
[iter 100] loss=0.4357 val_loss=0.4295 scale=0.5000 norm=0.9594
== Early stopping achieved.
== Best iteration / VAL1 (val_loss=0.4287)
[iter 0] loss=0.4321 val_loss=0.4440 scale=1.0000 norm=1.9078
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4440)
0.83831095
W

In [227]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6233 scale=2.0000 norm=4.0000
[iter 100] loss=0.4532 val_loss=0.4680 scale=1.0000 norm=1.8431
[iter 200] loss=0.4448 val_loss=0.4671 scale=0.0020 norm=0.0036
== Early stopping achieved.
== Best iteration / VAL136 (val_loss=0.4661)
[iter 0] loss=0.4532 val_loss=0.4335 scale=1.0000 norm=1.9267
[iter 100] loss=0.4435 val_loss=0.4343 scale=1.0000 norm=1.8965
== Early stopping achieved.
== Best iteration / VAL11 (val_loss=0.4333)
[iter 0] loss=0.4426 val_loss=0.4363 scale=0.5000 norm=0.9922
[iter 100] loss=0.4373 val_loss=0.4425 scale=0.5000 norm=0.9461
== Early stopping achieved.
== Best iteration / VAL5 (val_loss=0.4363)
[iter 0] loss=0.4404 val_loss=0.4299 scale=2.0000 norm=3.8692
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4299)
[iter 0] loss=0.4327 val_loss=0.4433 scale=1.0000 norm=1.9073
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4433)
0.8371852500000001
Wall time: 1min 32s


기존대로

In [234]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6266 scale=2.0000 norm=4.0000
[iter 100] loss=0.4627 val_loss=0.4736 scale=1.0000 norm=1.8355
== Early stopping achieved.
== Best iteration / VAL176 (val_loss=0.4680)
[iter 0] loss=0.4575 val_loss=0.4394 scale=1.0000 norm=1.8965
== Early stopping achieved.
== Best iteration / VAL44 (val_loss=0.4373)
[iter 0] loss=0.4487 val_loss=0.4454 scale=1.0000 norm=1.8987
== Early stopping achieved.
== Best iteration / VAL11 (val_loss=0.4451)
[iter 0] loss=0.4480 val_loss=0.4397 scale=1.0000 norm=1.9042
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4397)
[iter 0] loss=0.4437 val_loss=0.4544 scale=2.0000 norm=3.7880
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4544)
0.8336748


In [235]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6267 scale=2.0000 norm=4.0000
[iter 100] loss=0.4632 val_loss=0.4752 scale=2.0000 norm=3.6587
== Early stopping achieved.
== Best iteration / VAL186 (val_loss=0.4675)
[iter 0] loss=0.4557 val_loss=0.4362 scale=1.0000 norm=1.8963
== Early stopping achieved.
== Best iteration / VAL12 (val_loss=0.4360)
[iter 0] loss=0.4506 val_loss=0.4478 scale=1.0000 norm=1.8841
== Early stopping achieved.
== Best iteration / VAL18 (val_loss=0.4474)
[iter 0] loss=0.4495 val_loss=0.4415 scale=1.0000 norm=1.8957
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4415)
[iter 0] loss=0.4454 val_loss=0.4560 scale=1.0000 norm=1.8850
== Early stopping achieved.
== Best iteration / VAL1 (val_loss=0.4559)
0.8331747500000001


In [236]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6260 scale=2.0000 norm=4.0000
[iter 100] loss=0.4620 val_loss=0.4639 scale=2.0000 norm=3.6765
== Early stopping achieved.
== Best iteration / VAL161 (val_loss=0.4603)
[iter 0] loss=0.4523 val_loss=0.4584 scale=1.0000 norm=1.8638
== Early stopping achieved.
== Best iteration / VAL65 (val_loss=0.4551)
[iter 0] loss=0.4474 val_loss=0.4228 scale=1.0000 norm=1.8885
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4228)
[iter 0] loss=0.4454 val_loss=0.4360 scale=1.0000 norm=1.8787
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4360)
[iter 0] loss=0.4436 val_loss=0.4484 scale=1.0000 norm=1.8742
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4484)
[iter 0] loss=0.4442 val_loss=0.4400 scale=2.0000 norm=3.7606
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4400)
[iter 0] loss=0.4443 val_loss=0.4345 scale=1.0000 norm=1.8802
== Early stopping achieved.
== Best iteration / VAL1 (val_loss=0.4345)
[i

In [237]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6260 scale=2.0000 norm=4.0000
[iter 100] loss=0.4619 val_loss=0.4647 scale=2.0000 norm=3.6666
== Early stopping achieved.
== Best iteration / VAL159 (val_loss=0.4602)
[iter 0] loss=0.4517 val_loss=0.4570 scale=1.0000 norm=1.8625
== Early stopping achieved.
== Best iteration / VAL47 (val_loss=0.4546)
[iter 0] loss=0.4484 val_loss=0.4253 scale=1.0000 norm=1.8821
== Early stopping achieved.
== Best iteration / VAL12 (val_loss=0.4250)
[iter 0] loss=0.4458 val_loss=0.4355 scale=2.0000 norm=3.7447
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4355)
[iter 0] loss=0.4435 val_loss=0.4514 scale=1.0000 norm=1.8657
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4514)
[iter 0] loss=0.4446 val_loss=0.4391 scale=2.0000 norm=3.7499
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4391)
[iter 0] loss=0.4445 val_loss=0.4345 scale=1.0000 norm=1.8750
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4345)
[

In [238]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp2, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6262 scale=2.0000 norm=4.0000
[iter 100] loss=0.4613 val_loss=0.4753 scale=2.0000 norm=3.6862
== Early stopping achieved.
== Best iteration / VAL170 (val_loss=0.4710)
[iter 0] loss=0.4581 val_loss=0.4406 scale=2.0000 norm=3.8016
== Early stopping achieved.
== Best iteration / VAL34 (val_loss=0.4397)
[iter 0] loss=0.4510 val_loss=0.4445 scale=1.0000 norm=1.8956
== Early stopping achieved.
== Best iteration / VAL8 (val_loss=0.4442)
[iter 0] loss=0.4497 val_loss=0.4414 scale=1.0000 norm=1.8961
== Early stopping achieved.
== Best iteration / VAL2 (val_loss=0.4414)
[iter 0] loss=0.4452 val_loss=0.4563 scale=1.0000 norm=1.8852
== Early stopping achieved.
== Best iteration / VAL12 (val_loss=0.4561)
0.8322795000000001


In [239]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp3, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6262 scale=2.0000 norm=4.0000
[iter 100] loss=0.4628 val_loss=0.4770 scale=2.0000 norm=3.6779
== Early stopping achieved.
== Best iteration / VAL173 (val_loss=0.4718)
[iter 0] loss=0.4581 val_loss=0.4402 scale=1.0000 norm=1.9022
== Early stopping achieved.
== Best iteration / VAL23 (val_loss=0.4395)
[iter 0] loss=0.4522 val_loss=0.4467 scale=1.0000 norm=1.8916
== Early stopping achieved.
== Best iteration / VAL8 (val_loss=0.4466)
[iter 0] loss=0.4517 val_loss=0.4430 scale=1.0000 norm=1.8992
== Early stopping achieved.
== Best iteration / VAL3 (val_loss=0.4430)
[iter 0] loss=0.4468 val_loss=0.4583 scale=1.0000 norm=1.8873
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4583)
0.8311225500000001


In [240]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp2, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6254 scale=2.0000 norm=4.0000
[iter 100] loss=0.4627 val_loss=0.4651 scale=2.0000 norm=3.6877
== Early stopping achieved.
== Best iteration / VAL164 (val_loss=0.4603)
[iter 0] loss=0.4520 val_loss=0.4599 scale=1.0000 norm=1.8692
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4599)
[iter 0] loss=0.4542 val_loss=0.4350 scale=1.0000 norm=1.8783
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4350)
[iter 0] loss=0.4526 val_loss=0.4439 scale=2.0000 norm=3.7430
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4439)
[iter 0] loss=0.4508 val_loss=0.4525 scale=2.0000 norm=3.7350
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4525)
[iter 0] loss=0.4506 val_loss=0.4462 scale=1.0000 norm=1.8740
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4462)
[iter 0] loss=0.4504 val_loss=0.4433 scale=1.0000 norm=1.8740
== Early stopping achieved.
== Best iteration / VAL3 (val_loss=0.4433)
[it

In [241]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp3, train_b_p.target, 10)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6255 scale=2.0000 norm=4.0000
[iter 100] loss=0.4627 val_loss=0.4664 scale=2.0000 norm=3.6872
== Early stopping achieved.
== Best iteration / VAL152 (val_loss=0.4624)
[iter 0] loss=0.4533 val_loss=0.4608 scale=1.0000 norm=1.8664
== Early stopping achieved.
== Best iteration / VAL63 (val_loss=0.4586)
[iter 0] loss=0.4486 val_loss=0.4280 scale=1.0000 norm=1.8914
== Early stopping achieved.
== Best iteration / VAL20 (val_loss=0.4274)
[iter 0] loss=0.4459 val_loss=0.4345 scale=1.0000 norm=1.8820
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4345)
[iter 0] loss=0.4440 val_loss=0.4476 scale=1.0000 norm=1.8771
== Early stopping achieved.
== Best iteration / VAL1 (val_loss=0.4476)
[iter 0] loss=0.4447 val_loss=0.4378 scale=1.0000 norm=1.8850
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4378)
[iter 0] loss=0.4446 val_loss=0.4351 scale=1.0000 norm=1.8844
== Early stopping achieved.
== Best iteration / VAL0 (val_loss=0.4351)
[

In [261]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_ngb = np.mean(pred_y_list, axis = 0)

In [262]:
pred_ensemble_ngb

array([[0.99844489],
       [0.22853689],
       [0.47283505],
       ...,
       [0.84592395],
       [0.9229176 ],
       [0.34422363]])

In [263]:
submission.problem = pred_ensemble_ngb
submission

,user_id,problem
0,30000,0.998445
1,30001,0.228537
2,30002,0.472835
3,30003,0.860387
4,30004,0.993916
...,...,...
14994,44994,0.178637
14995,44995,0.247501
14996,44996,0.845924
14997,44997,0.922918


In [264]:
submission.to_csv(r'./submission/team_210203_2_ngbc-tuned.csv', index=False)

## Ensemble

In [281]:
pred_ensemble_catb

array([[0.91969449],
       [0.23163335],
       [0.51476496],
       ...,
       [0.68887411],
       [0.9339568 ],
       [0.41309126]])

In [282]:
pred_ensemble_lgbm

array([[0.93819144],
       [0.24549661],
       [0.44315962],
       ...,
       [0.76825373],
       [0.90830821],
       [0.42843415]])

In [283]:
pred_ensemble_ngb

array([[0.99844489],
       [0.22853689],
       [0.47283505],
       ...,
       [0.84592395],
       [0.9229176 ],
       [0.34422363]])

In [285]:
pred_ensemble_best = (
                            pred_ensemble_catb +
                            pred_ensemble_lgbm  +
                            pred_ensemble_ngb
                        ) / 3

submission.problem = pred_ensemble_best
submission

,user_id,problem
0,30000,0.952110
1,30001,0.235222
2,30002,0.476920
3,30003,0.827384
4,30004,0.939674
...,...,...
14994,44994,0.181496
14995,44995,0.274518
14996,44996,0.767684
14997,44997,0.921728


In [286]:
submission.to_csv(r'./submission/team_210203_3_ensemble-best3-final.csv', index=False)

In [76]:
from sklearn.ensemble import VotingClassifier

In [77]:
def vc_fold_train_pred(candidates, train_x, train_y):
    
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    oob_scores   = []
    threshold = 0.5
    
    s_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    vc = VotingClassifier(
                            estimators=[candidates],
                            voting='soft'
                        )
    
    for train_idx, val_idx in s_fold.split(train_x, train_y):
        
        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = vc.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)
        
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [79]:
vc = VotingClassifier(
                        estimators=[candidates],
                        voting='soft'
                    )

In [86]:
vc

VotingClassifier(estimators=[[('lgbm',
                               LGBMClassifier(random_state=2584, verbose=0)),
                              ('catb',
                               <catboost.core.CatBoostClassifier object at 0x00000211655B1F48>),
                              ('gbc',
                               GradientBoostingClassifier(random_state=2584)),
                              ('ngbc',
                               NGBClassifier(learning_rate=0.027,
                                             n_estimators=10000,
                                             random_state=RandomState(MT19937) at 0x20FBFD87598))]],
                 voting='soft')

In [78]:
candidates = [('lgbm', lgbm), ('catb', catb), ('gbc', gbc), ('ngbc', ngbc)]

models, auc_scores, _, _ = vc_fold_train_pred(candidates, X, train_b_p.target)
print(np.mean(auc_scores))

ValueError: too many values to unpack (expected 2)

In [146]:
VotingClassifier?

Init signature:
VotingClassifier(
    estimators,
    *,
    voting='hard',
    weights=None,
    n_jobs=None,
    flatten_transform=True,
    verbose=False,
)
Docstring:     
Soft Voting/Majority Rule classifier for unfitted estimators.

.. versionadded:: 0.17

Read more in the :ref:`User Guide <voting_classifier>`.

Parameters
----------
estimators : list of (str, estimator) tuples
    Invoking the ``fit`` method on the ``VotingClassifier`` will fit clones
    of those original estimators that will be stored in the class attribute
    ``self.estimators_``. An estimator can be set to ``'drop'``
    using ``set_params``.

    .. versionchanged:: 0.21
        ``'drop'`` is accepted.

    .. deprecated:: 0.22
       Using ``None`` to drop an estimator is deprecated in 0.22 and
       support will be dropped in 0.24. Use the string ``'drop'`` instead.

voting : {'hard', 'soft'}, default='hard'
    If 'hard', uses predicted class labels for majority rule voting.
    Else if 'soft', predict

# 창고

In [ ]:
# # 형록이가 제안한 var 모델

# X = pd.concat([train_e.user_id, pd.get_dummies(train_e.errtype)], axis=1).groupby('user_id').var().fillna(0)
# y = pd.concat([test_e.user_id, pd.get_dummies(test_e.errtype)], axis=1).groupby('user_id').var().fillna(0)

# X.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
#  'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
#  'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
#  'et_39', 'et_40', 'et_41', 'et_42']
# y.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
#  'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
#  'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
#  'et_39', 'et_40', 'et_41', 'et_42']

In [ ]:
# 혜인이가 제안한 errcode

# # 8호
# train_ecode_14 = train_e[train_e.errtype == 14][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_14 = test_e[test_e.errtype == 14][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_14.columns = ['err_14_1', 'err_14_13','err_14_14']
# test_ecode_14.columns =  ['err_14_1', 'err_14_13','err_14_14']

# # 30호
# train_ecode_30 = train_e[train_e.errtype == 30][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_30 = test_e[test_e.errtype == 30][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_30.columns = ['err_30_0','err_30_1','err_30_2', 'err_30_3','err_30_4']
# test_ecode_30.columns =  ['err_30_0','err_30_1','err_30_2', 'err_30_3','err_30_4']

# # 9호
# train_ecode_9 = train_e[train_e.errtype == 9][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_9 = test_e[test_e.errtype == 9][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_9.columns = ['err_9_1','err_9_C-11020', 'err_9_C-12032','err_9_C-13039','err_9_C-14014',
#                           'err_9_V-21002','err_9_V-21004','err_9_V-21005','err_9_V-21008']
# test_ecode_9.columns = ['err_9_1','err_9_C-11020', 'err_9_C-12032','err_9_C-13039','err_9_C-14014',
#                           'err_9_V-21002','err_9_V-21004','err_9_V-21005','err_9_V-21008']

# # 37호
# train_ecode_37 = train_e[train_e.errtype == 37][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_37 = test_e[test_e.errtype == 37][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_37.columns = ['err_37_1', 'err_37_2']
# test_ecode_37.columns = ['err_37_1', 'err_37_2']


# # 5호
# train_ecode_5 = train_e[train_e.errtype == 5][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)[('time', 'E-59902')]
# test_ecode_5 = test_e[test_e.errtype == 5][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)[('time', 'E-59902')]
# #train_ecode_5.columns = ['err_5_E-59902']
# #test_ecode_5.columns = ['err_5_E-59902']
# train_ecode_5 = train_ecode_5.rename(level = 0, index = 'err_5_E-59902') 
# test_ecode_5 = test_ecode_5.rename(level = 0, index = 'err_5_E-59902') 

# # 31호
# train_ecode_31 = train_e[train_e.errtype == 31][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_31 = test_e[test_e.errtype == 31][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_31.columns = ['err_31_0', 'err_31_1']
# test_ecode_31.columns =['err_31_0', 'err_31_1']

# # 7호
# train_ecode_7 = train_e[train_e.errtype == 7][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_7 = test_e[test_e.errtype == 7][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_7.columns = ['err_7_1', 'err_7_14']
# test_ecode_7.columns =['err_7_1', 'err_7_14']

# # 25호
# train_ecode_25 = train_e[train_e.errtype == 25][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_25 = test_e[test_e.errtype == 25][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

# train_ecode_25.columns =  ['err_25_1', 'err_25_2', 'err_25_L2CAP connection cancelled', 'err_25_UNKNOWN', 
#                             'err_25_connection fail to establish',
#                              'err_25_connection timeout', 'err_25_connectionterminated by local host',
#                             'err_25_scanning timeout', 'err_25_terminate by peer user']

# test_ecode_25.columns = ['err_25_1', 'err_25_2', 'err_25_L2CAP connection cancelled', 'err_25_UNKNOWN', 
#                            'connection fail for LMP response timout',
#                             'err_25_connection fail to establish',
#                              'err_25_connection timeout', 'err_25_connectionterminated by local host',
#                             'err_25_scanning timeout', 'err_25_terminate by peer user']

# test_ecode_25.drop('connection fail for LMP response timout', axis=1, inplace=True)

### Feature Selection: Ridge, Lasso

In [ ]:
# from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import roc_auc_score

In [ ]:
# scaler = MinMaxScaler()

In [ ]:
# X_new_scaled = scaler.fit_transform(X_new)

In [ ]:
# a_list = [0.00001, 0.0001, 0.001, 0.01,
#           0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9]

In [ ]:
# # Ridge
# ridge = RidgeCV(alphas=a_list)
# ridge.fit(X_new_scaled, train_b_p.target)

# print('best alpha: ', ridge.alpha_)
# print('feature selected: ', ridge.coef_.nonzero()[0].size)
# print('R^2 is: ', ridge.score(X_new_scaled, train_b_p.target))
# print('auc: ', roc_auc_score(train_b_p.target, ridge.predict(X_new_scaled)))

In [ ]:
# # Lasso
# lasso = LassoCV(alphas=a_list)
# lasso.fit(X_new_scaled, train_b_p.target)

# print('best alpha: ', lasso.alpha_)
# print('feature selected: ', lasso.coef_.nonzero()[0].size)
# print('R^2 is: ', lasso.score(X_new_scaled, train_b_p.target))
# print('auc: ', roc_auc_score(train_b_p.target, lasso.predict(X_new_scaled)))

In [ ]:
# # elastic
# elastic = ElasticNetCV(alphas=a_list)
# elastic.fit(X_new_scaled, train_b_p.target)

# print('best alpha: ', elastic.alpha_)
# print('feature selected: ', elastic.coef_.nonzero()[0].size)
# print('R^2 is: ', elastic.score(X_new_scaled, train_b_p.target))
# print('auc: ', roc_auc_score(train_b_p.target, elastic.predict(X_new_scaled)))

In [ ]:
# elastic.coef_.nonzero()[0]

In [ ]:
# X_new.iloc[:, elastic.coef_.nonzero()[0]].columns

In [ ]:
# X_selected = X_new.iloc[:, elastic.coef_.nonzero()[0]]

### Base LGBM

In [117]:
# #------------------------------------------------------------
# # validation auc score를 확인하기 위해 정의
# def f_pr_auc(probas_pred, y_true):
#     labels=y_true.get_label()
#     p, r, _ = precision_recall_curve(labels, probas_pred)
#     score=auc(r,p) 
#     return "pr_auc", score, True
# #------------------------------------------------------------


# def s_fold_train_pred(train_x, train_y):
#     import lightgbm as lgb

#     # Train
#     models     = []
#     recalls    = []
#     precisions = []
#     auc_scores   = []
#     threshold = 0.5
#     # 파라미터 설정
#     params =      {
#                     'boosting_type' : 'gbdt',
#                     'objective'     : 'binary',
#                     'metric'        : 'auc',
#                     'learning_rate' : 0.027,
#                     'seed': 42
#                     }
#     #-------------------------------------------------------------------------------------
#     # 5 Kfold cross validation
#     s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

#     for train_idx, val_idx in s_fold.split(train_x, train_y):

#         # split train, validation set
#         X = train_x.iloc[train_idx]
#         y = train_y.iloc[train_idx]
#         valid_x = train_x.iloc[val_idx]
#         valid_y = train_y.iloc[val_idx]

#         d_train= lgb.Dataset(X, y)
#         d_val  = lgb.Dataset(valid_x, valid_y)

#         #run traning
#         model = lgb.train(
#                             params,
#                             train_set       = d_train,
#                             num_boost_round = 10000,
#                             valid_sets      = d_val,
#                             feval           = f_pr_auc,
#                             verbose_eval    = 100, 
#                             early_stopping_rounds = 100
#                            )

#         # cal valid prediction
#         valid_prob = model.predict(valid_x)
#         valid_pred = np.where(valid_prob > threshold, 1, 0)

#         # cal scores
#         recall    = recall_score(    valid_y, valid_pred)
#         precision = precision_score( valid_y, valid_pred)
#         auc_score = roc_auc_score(   valid_y, valid_prob)

#         # append scores
#         models.append(model)
#         recalls.append(recall)
#         precisions.append(precision)
#         auc_scores.append(auc_score)

#         print('==========================================================')
        
#     return models, auc_scores, recalls, precisions

In [52]:
# # loss 비교를 위해 지우지 않습니다.
# # 최고점 모델입니다.
# models, auc_scores, _, _ = s_fold_train_pred(X, train_b_p.target)
# print(np.mean(auc_scores))

In [53]:
# # 학습용
# models, auc_scores, _, _ = s_fold_train_pred(X_temp, train_b_p.target)
# print(np.mean(auc_scores))

### Tuned Xtree

In [165]:
# 파라미터 설정
xtree = ExtraTreesClassifier(
                                bootstrap=False,
                                ccp_alpha=0.0,
                                class_weight=None,
                                criterion='gini',
                                max_depth=None,
                                max_features='auto',
                                max_leaf_nodes=None,
                                max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None,
                                min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, 
                                n_estimators=2000,
                                n_jobs=-1,
                                oob_score=False,
                                random_state=2584,
                                verbose=1,
                                warm_start=False
                            )

In [166]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------


def xtree_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5

    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = xtree
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

        print('==========================================================')
        
    return models, auc_scores, recalls, precisions

In [167]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = xtree_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.5s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    4.2s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    5.1s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    4.5s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

0.7254


In [168]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = xtree_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    5.4s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s


0.7257


[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_jobs=24)]: Done 2000 out of 2000 | elapsed:    0.3s finished


In [78]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_xtree = np.mean(pred_y_list, axis = 0)

[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.3s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.5s
[Parallel(n_jobs=24)]: Done 2000 out of 2000 | elapsed:    0.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.3s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.5s
[Parallel(n_job

In [79]:
pred_ensemble_xtree

array([[0.9055],
       [0.1745],
       [0.4875],
       ...,
       [0.603 ],
       [0.6785],
       [0.458 ]])

### Tuned RF

In [80]:
rfc = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=7367, verbose=0,
                       warm_start=False)

In [81]:
def rf_fold_train_pred(train_x, train_y):
    
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    oob_scores   = []
    threshold = 0.5
    
    s_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  
    
    for train_idx, val_idx in s_fold.split(train_x, train_y):
        
        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = rfc.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)
        
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [82]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = rf_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.7261500000000002


In [125]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = rf_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.7283000000000001


In [83]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_rf = np.mean(pred_y_list, axis = 0)

In [84]:
pred_ensemble_rf

array([[0.82],
       [0.2 ],
       [0.49],
       ...,
       [0.51],
       [0.66],
       [0.34]])

### Tuned GBC

In [202]:
gbc = GradientBoostingClassifier(
                                ccp_alpha=0.0,
                                criterion='friedman_mse',
                                init=None,
                                learning_rate=0.1,
                                loss='deviance',
                                max_depth=3,
                                max_features=None,
                                max_leaf_nodes=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None,
                                min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0,
                                n_estimators=1000,
                                n_iter_no_change=None,
                                presort='deprecated',
                                random_state=2584,
                                subsample=1.0,
                                tol=0.0001,
                                validation_fraction=0.1,
                                verbose=0,
                                warm_start=False
                            )

In [203]:
def gbc_fold_train_pred(train_x, train_y, N_SPLIT=5):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5

    k_fold = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
    for train_idx, val_idx in k_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = gbc.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)
        
        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
    
    return models, auc_scores, recalls, precisions

검증 중

In [204]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp2, train_b_p.target, 10)
print(np.mean(auc_scores))

KeyboardInterrupt: 

In [205]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp3, train_b_p.target, 10)
print(np.mean(auc_scores))

KeyboardInterrupt: 

In [ ]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp2, train_b_p.target)
print(np.mean(auc_scores))

In [ ]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp3, train_b_p.target)
print(np.mean(auc_scores))

이터레이션 100-> 1000으로 증가

In [145]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X, train_b_p.target, 10)
print(np.mean(auc_scores))

0.7314


In [156]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp, train_b_p.target, 10)
print(np.mean(auc_scores))

0.73225
Wall time: 14min 23s


In [172]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.7308999999999999
Wall time: 5min 59s


In [173]:
%%time
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.7323500000000001
Wall time: 6min 18s


이터레이션 100 그대로

In [94]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.7252500000000001


In [95]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.72435


In [64]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

0.72455


In [65]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.72495


In [70]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_gbc = np.mean(pred_y_list, axis = 0)

In [71]:
pred_ensemble_gbc

array([[0.94729912],
       [0.20899576],
       [0.48470505],
       ...,
       [0.75559137],
       [0.87481054],
       [0.34822194]])